# What is LangGraph?

1. It is an orcgestration framework for building **intelligent, stateful and multi-step** LLM workflows.
2. It Enables Advance Features like **Parallelism, loops, branching, memory and resumability** making it an ideal candidate and production-grade AI Application.
3. It models your **logic as a graph of nodes(task)** and **edges (routing)** instead of linear chain.

## 1.1 What is LLM Workflow?

1. LLM workflows are a step by step process which we can build complex LLM application
2. Each step in the workflow performs a distinct task - Such as Prompting, reasoning, tool callling, memory access, or decision making
3. Workflow can be linear, parallel, branched or looped, allowing for complex behaviours like retries, multi-agent communication or tool augmented reasoning

## 2. Common Workflows
![alt text](./agent_workflow.png)

## 3.2 Graphs, Nodes and Edges

At its core, LangGraph models agent workflows as graphs. You define the behavior of your agents using three key components:

    Nodes: Functions that encode the logic of your agents. They receive the current state as input, perform some computation or side-effect, and return an updated state.
    Edges: Functions that determine which Node to execute next based on the current state. They can be conditional branches or fixed transitions.
    State: A shared data structure that represents the current snapshot of your application. It can be any data type, but is typically defined using a shared state schema.

By composing Nodes and Edges, you can create complex, looping workflows that evolve the state over time. The real power, though, comes from how LangGraph manages that state. To emphasize: Nodes and Edges are nothing more than functions – they can contain an LLM or just good ol’ code. In short: nodes do the work, edges tell what to do next. 

```
LangGraph’s underlying graph algorithm uses message passing to define a general program. When a Node completes its operation, it sends messages along one or more edges to other node(s). These recipient nodes then execute their functions, pass the resulting messages to the next set of nodes, and the process continues. Inspired by Google’s Pregel system, the program proceeds in discrete “super-steps.”
```

```
A super-step can be considered a single iteration over the graph nodes. Nodes that run in parallel are part of the same super-step, while nodes that run sequentially belong to separate super-steps. At the start of graph execution, all nodes begin in an inactive state. A node becomes active when it receives a new message (state) on any of its incoming edges (or “channels”). The active node then runs its function and responds with updates. At the end of each super-step, nodes with no incoming messages vote to halt by marking themselves as inactive. The graph execution terminates when all nodes are inactive and no messages are in transit.
```

### 3.2.1 Nodes


In LangGraph, nodes are Python functions (either synchronous or asynchronous) that accept the following arguments:

    1. state – The state of the graph
    2. config – A RunnableConfig object that contains configuration information like thread_id and tracing information like tags
    3. runtime – A Runtime object that contains runtime context and other information like store and stream_writer

Similar to NetworkX, you add these nodes to a graph using the add_node method:

#### START node
The START Node is a special node that represents the node that sends user input to the graph. The main purpose for referencing this node is to determine which nodes should be called first.

```python
    from langgraph.graph import START

    graph.add_edge(START, "node_a")
```

#### END node
The END Node is a special node that represents a terminal node. This node is referenced when you want to denote which edges have no actions after they are done.

```python
    from langgraph.graph import END
    
    graph.add_edge("node_a", END)
```


#### Node caching
LangGraph supports caching of tasks/nodes based on the input to the node. To use caching:

    * Specify a cache when compiling a graph (or specifying an entrypoint)
    * Specify a cache policy for nodes. Each cache policy supports:
        - key_func used to generate a cache key based on the input to a node, which defaults to a hash of the input with pickle.
        - ttl, the time to live for the cache in seconds. If not specified, the cache will never expire.


## 3.3 State

The first thing you do when you define a graph is define the **State of the graph**. The State consists of the schema of the graph as well as reducer functions which specify how to apply updates to the state. The schema of the State will be the input schema to all Nodes and Edges in the graph, and can be either a **TypedDict or a Pydantic** model. All Nodes will emit updates to the State which are then applied using the specified reducer function.

## 3.4 Reducer
Reducers are key to understanding how updates from nodes are applied to the State. **Each key in the State has its own independent reducer function**. If no reducer function is explicitly specified then it is assumed that all updates to that key should override it. There are a few different types of reducers, starting with the default type of reducer:

## 4 LangGraph Execution Model

*1-Graph Definition*



You define:



- The state schema

- Nodes

- Edges



*2-Compilation*



You call `.compile()` on the `StateGraph`



This checks the graph structure and prepares it for execution



*3-Invocation*



You run the graph with `.invoke(intial_state)`



LangGraph sends the initial state as a message to the entry node(s).



*4-Super step Begin*



Execution proceeds in rounds



*5-Message passing & Node activation*



The messages are passed to downstream nodes via edges



Nodes that receive messages become active for the next round



*6-Halting condition*



Execution stops when



- No nodes are active, and

- No messages are in transit